### Inside the heads of Putnuts
#### An analysis of the Only Murders in the Building TV show subreddit

Milindi Kodikara

#### Step 1 : : Set up

In [ ]:
from client import client
import helper


import praw
import string
import json
import codecs
import re

import nltk 
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

from collections import Counter

# misc
import datetime as dt
from pprint import pprint
from itertools import chain
from colorama import Fore, Back, Style
import pandas as pd

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 8) # default plot size
import seaborn as sns
sns.set(style='whitegrid', palette='Dark2')
from wordcloud import WordCloud

In [ ]:
subreddit_name = 'OnlyMurdersHulu'
# maximum number of hot submissions
# TODO: Reset to 100 for analysis
limit = 1

In [ ]:
reddit_client = client()

In [ ]:
# sanity check
print('Username :: ', reddit_client.user.me())

In [ ]:
subreddit = reddit_client.subreddit(subreddit_name)

print('Subreddit :: ', subreddit)

In [ ]:
posts = [*subreddit.hot(limit=limit)] 

len(posts)

In [ ]:
for submission in posts:
    print('Title: {}\nAuthor: {}\nDescription: {}\nUpvote ratio: {}\n---------'.format(submission.title, submission.author.name, submission.selftext, submission.upvote_ratio))

In [ ]:
term_freq = 50
term_freq_counter = Counter()

token_list_list = []
for submission in subreddit.hot(limit=limit):
    post_description = submission.selftext
    post_title = submission.title
    
    post_title_description = post_title + " " + post_description
    
    # text, tokeniser, stop_words
    token_list_list.append(TweetTokenizer().tokenize(post_title_description))
    
    submission.comments.replace_more(limit=None)
    for comment in submission.comments:
        comment_text = comment.body
        
        token_list_list.append(TweetTokenizer().tokenize(comment_text))

flatted_list = [element for innerList in token_list_list for element in innerList]        
term_freq_counter.update(flatted_list)


print("-----------------\nUnprocessed data term frequency\n-----------------\n")
for term, count in term_freq_counter.most_common(term_freq):
    print(term + ': ' + str(count))

y = [count for term, count in term_freq_counter.most_common(term_freq)]
x = [term for term, count in term_freq_counter.most_common(term_freq)]

# use matplotlib bar chat to plot this
plt.bar(x, y, color='#275c4d')
plt.title("Term frequency distribution")
plt.ylabel('# of words with term frequency')
plt.xlabel('Term frequency')
plt.xticks(rotation=90)

plt.show()

In [ ]:
# TODO: posts per date

In [ ]:
# TODO: posts per user

In [ ]:
# TODO: comments per post

In [ ]:
# TODO: comments vs upvotes 

In [ ]:
# TODO: language breakdown

In [ ]:
# TODO: posts based on location

In [ ]:
# TODO: Post authors' commenting freq

#### Step 2 : : Pre-processing

In [ ]:
# TODO: Ask why use Tweet Tokenizer, why not word_tokenizer
tokeniser = TweetTokenizer()
# add punctuation to stopwords list
stop_words = stopwords.words('english') + list(string.punctuation) + ['rt', 'via', '...', '…', '"', "'", '`']

In [ ]:
# Tokenize and pre-process -> Add to a new dataframe to be analysed 
token_list = []
for submission in subreddit.hot(limit=limit):
    post_description = submission.selftext
    post_title = submission.title
    
    post_title_description = post_title + " " + post_description
    
    # text, tokeniser, stop_words
    token_list.append(helper.process(post_title_description, tokeniser, stop_words))
    
    comment_token_list = []
    submission.comments.replace_more(limit=None)
    for comment in submission.comments:
        comment_text = comment.body
        
        comment_token_list.append(helper.process(comment_text, tokeniser, stop_words))
    
    print('Title: {}\nAuthor: {}\nDescription: {}\n'.format(submission.title, submission.author.name, submission.selftext))
    print('Token list for post: {}\nfor comments: {} \n-----------------\n'.format(token_list, comment_token_list))

In [ ]:
# TODO: Generate top k unigrams pre and post clean

In [ ]:
# TODO: Generate top k bigrams pre and post clean

#### Step 3 : : Models

1. n-grams
2. upvotes
3. sentiment analysis 
    - count method
        TODO: <sentiment / dates>
    - vader
        TODO: <sentiment / dates>
4. topic modelling
    - term doc freq
    - TDA
    - params


In [ ]:
# sentiment_analysis = SentimentIntensityAnalyzer()

# sentiment_scores = sentiment_analysis.polarity_scores(" ".join(token_list))

In [ ]:
# https://medium.com/bitgrit-data-science-publication/sentiment-analysis-on-reddit-tech-news-with-python-cbaddb8e9bb6

#### Step 4 : : Analysis

1. Present what topics are being discussed eg: top-K terms, word-cloud etc. by **topic modelling** 
2. What are the topics, does it correspond to recent news etc